In [1]:
# Standard library imports
import os
from typing import Literal

# Third-party imports
import spacy
import trafilatura
import ibm_db
import ibm_db_dbi
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from IPython.display import Image, display, Markdown

# LangChain core imports
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.tools.retriever import create_retriever_tool
from langchain_core.documents import Document
from langchain_core.messages import convert_to_messages
from langchain_core.vectorstores import InMemoryVectorStore

# LangChain community imports
from langchain_community.llms import LlamaCpp
from langchain.embeddings import LlamaCppEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

# LangChain IBM imports
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams, EmbedTextParamsMetaNames

# LangChain DB2 imports
from langchain_db2 import db2vs
from langchain_db2.db2vs import DB2VS

# LangGraph imports
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

# Load environment variables with explicit path and override
load_dotenv(os.path.join(os.getcwd(), ".env"), override=True)

True

In [2]:
DB_NAME = os.getenv("DB_NAME", "")
DB_HOST = os.getenv("DB_HOST", "")
DB_PORT = os.getenv("DB_PORT", "")
DB_PROTOCOL = os.getenv("DB_PROTOCOL", "")
DB_USER = os.getenv("DB_USER", "")
DB_PASSWORD = os.getenv("DB_PASSWORD", "")
LLM_PATH = os.getenv("LLM_PATH", "")
EMBEDDING_MODEL_PATH = os.getenv("EMBEDDING_MODEL_PATH", "")


conn_str=f"DATABASE={DB_NAME};hostname={DB_HOST};port={DB_PORT};protocol={DB_PROTOCOL};uid={DB_USER};pwd={DB_PASSWORD}"

try:
    connection = ibm_db_dbi.connect(conn_str, '', '')
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")

Connection successful!


## 1. Load and Preprocess documents

In [3]:
url = 'https://community.ibm.com/community/user/blogs/shaikh-quader/2024/05/07/building-an-in-db-linear-regression-model-with-ibm'
downloaded = trafilatura.fetch_url(url)

if downloaded:
    article = trafilatura.extract(downloaded)
    print(article[:1000])  # Preview first 1000 chars
else:
    print("Failed to fetch content.")

Despite being one of the earlier machine learning techniques, linear regression continues to be a top choice among ML practitioners for a regression task. For the past three years, over 80% of the respondents to Kaggle’s annual state of data science and machine learning survey mentioned linear regression as a ML algorithm they most frequently use. IBM Db2 provides an in-database stored procedure (SP) for Linear Regression as part of its ML library, which is a collection of over 200 SPs for performing different ML tasks in the database. Using the linear regression SP and other functionality of DB2’s ML Library, ML practitioners can build and deploy linear regression models in the database when their ML dataset is available in a Db2 database. In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:
Let’s begin.
The Regression Task
In this exercise, I will use the GoSales dataset, which is available from this link.

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
def overlapping_sentence_chunker(text, max_words=200, overlap_words=50):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    
    chunks = []
    current_chunk = []
    current_length = 0

    i = 0
    while i < len(sentences):
        sentence = sentences[i]
        sentence_length = len(sentence.split())

        if current_length + sentence_length <= max_words:
            current_chunk.append(sentence)
            current_length += sentence_length
            i += 1
        else:
            chunks.append(" ".join(current_chunk))
            # Start new chunk with overlap
            overlap = []
            overlap_len = 0
            j = len(current_chunk) - 1
            while j >= 0 and overlap_len < overlap_words:
                s = current_chunk[j]
                overlap.insert(0, s)
                overlap_len += len(s.split())
                j -= 1
            current_chunk = overlap
            current_length = overlap_len

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [6]:
chunks = overlapping_sentence_chunker(article, max_words=200, overlap_words=50)
print(f"{len(chunks)} chunks created.")
print(chunks[0])

15 chunks created.
Despite being one of the earlier machine learning techniques, linear regression continues to be a top choice among ML practitioners for a regression task. For the past three years, over 80% of the respondents to Kaggle’s annual state of data science and machine learning survey mentioned linear regression as a ML algorithm they most frequently use. IBM Db2 provides an in-database stored procedure (SP) for Linear Regression as part of its ML library, which is a collection of over 200 SPs for performing different ML tasks in the database. Using the linear regression SP and other functionality of DB2’s ML Library, ML practitioners can build and deploy linear regression models in the database when their ML dataset is available in a Db2 database. In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:
Let’s begin. The Regression Task
In this exercise, I will use the GoSales dataset, which is availa

In [7]:
chunks

['Despite being one of the earlier machine learning techniques, linear regression continues to be a top choice among ML practitioners for a regression task. For the past three years, over 80% of the respondents to Kaggle’s annual state of data science and machine learning survey mentioned linear regression as a ML algorithm they most frequently use. IBM Db2 provides an in-database stored procedure (SP) for Linear Regression as part of its ML library, which is a collection of over 200 SPs for performing different ML tasks in the database. Using the linear regression SP and other functionality of DB2’s ML Library, ML practitioners can build and deploy linear regression models in the database when their ML dataset is available in a Db2 database. In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:\nLet’s begin. The Regression Task\nIn this exercise, I will use the GoSales dataset, which is available from this l

## 2. Create a retriever tool

Set up embedding model

In [8]:
%%capture init_logs

# embeddings = LlamaCppEmbeddings(model_path=EMBEDDING_MODEL_PATH)
embeddings = LlamaCppEmbeddings(
    model_path=EMBEDDING_MODEL_PATH,
    verbose=False
)

Set up vector store

In [9]:
vectorstore = DB2VS.from_texts(
    chunks,
    embeddings,
    client=connection,
    table_name="Documents_EUCLIDEAN",
    distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings requ

# Set up LLM

In [10]:
%%capture init_logs

llm = LlamaCpp(
    model_path=LLM_PATH,
    n_gpu_layers=0,
    n_threads=30,
    n_batch=512,
    max_tokens=250,       
    n_ctx=2048,
    seed=42,
    temperature=0.3,
    top_p=0.9,
    top_k=40,
    repeat_penalty=1.1,
    verbose=False
)

# The RAG Prompt

In [11]:
# Better prompt that encourages combining information
template = """<|system|>
You are a helpful assistant. When answering questions, synthesize information from all the provided context to give a comprehensive, well-structured response.<|end|>
<|user|>
Context Information:
{context}

Question: {question}

Instructions: 
- Use ALL relevant information from the context above
- Combine related points into a coherent answer
- Organize your response clearly with examples when available
- If multiple sources mention the same topic, integrate them smoothly<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [12]:
# Modify your RAG to return source documents
rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    verbose=False,
    return_source_documents=True  # This returns the retrieved context
)

In [13]:
result = rag.invoke('How to build a linear regression model in Db2?')

markdown_output = f"""
## 💡 Answer

{result['result']}

---

## 📚 Retrieved Context

"""

for i, doc in enumerate(result['source_documents'], 1):
    markdown_output += f"\n**📄 Chunk {i}**\n\n{doc.page_content}\n\n---\n"

display(Markdown(markdown_output))

init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding



## 💡 Answer

 To build a linear regression model in Db2, follow these steps:

1. **Prepare the Dataset**: Ensure that your dataset is clean and ready for analysis. In this case, we have already loaded the GoSales dataset into a Db2 table named GOSALES_FULL.

2. **Split the Data**: Divide the records from the GOSALES_FULL table into two partitions: a training partition and a test partition. This process ensures that the model is trained on data not used for testing, which helps in evaluating the performance of the model more accurately. The following SQL query can be used to split the data:

```sql
CALL IDAX.SPLIT_DATA('GOSALES_FULL', 'GOSALES.LINREG_TRAIN', 'GOSALES.LINREG_TEST');
```

3. **Train the Linear Regression Model**: Use the training partition of the dataset to train a linear regression model. The following SQL query can be used to create and train the linear regression model:

```sql
CALL IDAX.CREATE_MODEL('model=GOSALES.GOSALES_LINREG', 'algorithm=linear_regression', 'target_column=MAPE', 'training_data_partition=GOSALES.LINREG_TRAIN');
```

4. **Evaluate the Model

---

## 📚 Retrieved Context


**📄 Chunk 1**

Despite being one of the earlier machine learning techniques, linear regression continues to be a top choice among ML practitioners for a regression task. For the past three years, over 80% of the respondents to Kaggle’s annual state of data science and machine learning survey mentioned linear regression as a ML algorithm they most frequently use. IBM Db2 provides an in-database stored procedure (SP) for Linear Regression as part of its ML library, which is a collection of over 200 SPs for performing different ML tasks in the database. Using the linear regression SP and other functionality of DB2’s ML Library, ML practitioners can build and deploy linear regression models in the database when their ML dataset is available in a Db2 database. In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:
Let’s begin. The Regression Task
In this exercise, I will use the GoSales dataset, which is available from this link. The dataset has 60252 synthetic customers’ profile and their purchase amount at an imaginary outdoor equipment store.

---

**📄 Chunk 2**

In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:
Let’s begin. The Regression Task
In this exercise, I will use the GoSales dataset, which is available from this link. The dataset has 60252 synthetic customers’ profile and their purchase amount at an imaginary outdoor equipment store. The following is the list of columns in the dataset:
I want to learn a multiple linear regression function of the following equation form that will use as input the first four columns — AGE, MARITAL_STATUS, and PROFESSION — and predict the PURCHASE_AMOUNT. Using the training examples, the linear regression algorithm will learn the values of the following five parameters — the first one is the intercept and the remaining are four coefficients, one per input column. Following these steps, I created a Db2 table with the name GOSALES_FULL under GOSALES schema and loaded the dataset into it. Train / Test Split
First, I will divide the records from the GOSALES_FULL table into two partitions: a training partition and a test partition. For making these partitions, I will call the SPLIT_DATA stored procedure (SP) as follows.

---

**📄 Chunk 3**

A.PURCHASE_AMOUNT * 100) AS MAPE FROM GOSALES.GOSALES_TEST AS A, GOSALES.GOSALES_TEST_PREDICTIONS AS B WHERE A.ID = B.ID
Dropping the Model
If I want to drop this model, I can use the DROP_MODEL SP:
CALL IDAX.DROP_MODEL('model=GOSALES.GOSALES_LINREG')
Conclusion
In this exercise, I built, evaluated, and deployed an end-to-end linear regression pipeline using 29 simple SQL queries — 17 of them are calls to SPs provided by Db2, and the remaining ones were SELECT statements. In this ML workflow, I didn’t need to bring any data outside of the database, nor did I need separate infrastructure for developing, training, and serving ML models. For many companies, machine learning in the database can be a cost effective and quicker path to embrace AI.
Related Resources
For creating an in-database ML pipeline using my above steps, you can download the GoSales dataset from this link and follow instructions on this page to set up a Db2 database. To learn more about Db2’s in-database ML library, check out Db2 production documentation. To find more examples of creating in-database ML models with Db2, check out this GitHub repo. #Featured-area-2

---


In [14]:
result = rag.invoke('How to evaluate a linear regression model in Db2?')

markdown_output = f"""
## 💡 Answer

{result['result']}

---

## 📚 Retrieved Context

"""

for i, doc in enumerate(result['source_documents'], 1):
    markdown_output += f"\n**📄 Chunk {i}**\n\n{doc.page_content}\n\n---\n"

display(Markdown(markdown_output))

init: embeddings required but some input tokens were not marked as outputs -> overriding
init: embeddings required but some input tokens were not marked as outputs -> overriding



## 💡 Answer

 To evaluate a linear regression model in Db2, you can follow these steps:

1. Split the data into training and testing sets using the SPLIT_DATA stored procedure (SP).

CALL IDAX.SPLIT_DATA('table=GOSALES.GOSALES_FULL', 'column=ID', 'split_column=IS_TEST', 'training_set_percentage=70')

2. Train the linear regression model on the training set.

3. Evaluate the trained model's performance on the testing set using metrics such as Mean Absolute Error (MAE), Mean Squared Error (MSE), and Root Mean Squared Error (RMSE).

4. Use these evaluation metrics to make informed decisions about whether to use or improve the linear regression model in Db2.

By following these steps, you can effectively evaluate a linear regression model in Db2 using SQL with a Db2 database.<|end|>

---

## 📚 Retrieved Context


**📄 Chunk 1**

Despite being one of the earlier machine learning techniques, linear regression continues to be a top choice among ML practitioners for a regression task. For the past three years, over 80% of the respondents to Kaggle’s annual state of data science and machine learning survey mentioned linear regression as a ML algorithm they most frequently use. IBM Db2 provides an in-database stored procedure (SP) for Linear Regression as part of its ML library, which is a collection of over 200 SPs for performing different ML tasks in the database. Using the linear regression SP and other functionality of DB2’s ML Library, ML practitioners can build and deploy linear regression models in the database when their ML dataset is available in a Db2 database. In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:
Let’s begin. The Regression Task
In this exercise, I will use the GoSales dataset, which is available from this link. The dataset has 60252 synthetic customers’ profile and their purchase amount at an imaginary outdoor equipment store.

---

**📄 Chunk 2**

In this post, I will show you the following steps of building and using a linear regression pipeline using SQL with a Db2 database:
Let’s begin. The Regression Task
In this exercise, I will use the GoSales dataset, which is available from this link. The dataset has 60252 synthetic customers’ profile and their purchase amount at an imaginary outdoor equipment store. The following is the list of columns in the dataset:
I want to learn a multiple linear regression function of the following equation form that will use as input the first four columns — AGE, MARITAL_STATUS, and PROFESSION — and predict the PURCHASE_AMOUNT. Using the training examples, the linear regression algorithm will learn the values of the following five parameters — the first one is the intercept and the remaining are four coefficients, one per input column. Following these steps, I created a Db2 table with the name GOSALES_FULL under GOSALES schema and loaded the dataset into it. Train / Test Split
First, I will divide the records from the GOSALES_FULL table into two partitions: a training partition and a test partition. For making these partitions, I will call the SPLIT_DATA stored procedure (SP) as follows.

---

**📄 Chunk 3**

A.PURCHASE_AMOUNT * 100) AS MAPE FROM GOSALES.GOSALES_TEST AS A, GOSALES.GOSALES_TEST_PREDICTIONS AS B WHERE A.ID = B.ID
Dropping the Model
If I want to drop this model, I can use the DROP_MODEL SP:
CALL IDAX.DROP_MODEL('model=GOSALES.GOSALES_LINREG')
Conclusion
In this exercise, I built, evaluated, and deployed an end-to-end linear regression pipeline using 29 simple SQL queries — 17 of them are calls to SPs provided by Db2, and the remaining ones were SELECT statements. In this ML workflow, I didn’t need to bring any data outside of the database, nor did I need separate infrastructure for developing, training, and serving ML models. For many companies, machine learning in the database can be a cost effective and quicker path to embrace AI.
Related Resources
For creating an in-database ML pipeline using my above steps, you can download the GoSales dataset from this link and follow instructions on this page to set up a Db2 database. To learn more about Db2’s in-database ML library, check out Db2 production documentation. To find more examples of creating in-database ML models with Db2, check out this GitHub repo. #Featured-area-2

---
